# Algorithm Implement

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import quantstats as qs
import pandas as pd
import numpy as np
import sys
time_period = 2
sys.path.append('./')
from utlis import get_data, Stock_Env
import warnings
warnings.filterwarnings('ignore')
from collections import deque

class Q_Network(nn.Module):

    def __init__(self, state_size, action_size, hidden=[64, 64]):
        super(Q_Network, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size)

    def forward(self, state):
        x = state
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Data Loading

In [2]:
stock_df_train, stock_df_test, stock_df_train_, stock_df_test_, codes = get_data()

100%|███████████████████████████████████████████████████████████████████████████████| 252/252 [00:01<00:00, 189.98it/s]


# Technical Indicators

# RL

In [3]:
codes_dict = dict(zip(codes, range(len(codes))))

In [4]:
tst = None
import random
from collections import deque
import torch
import torch.optim as optim
import numpy as np

# from networks import *

class Agent:

    def __init__(self, state_size, action_size, bs, lr, tau, gamma, device):
        '''
        When dealing with visual inputs, state_size should work as num_of_frame
        '''
        self.state_size = state_size
        self.action_size = action_size
        self.bs = bs
        self.lr = lr
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.Q_local = Q_Network(self.state_size, self.action_size).to(device)
        self.Q_target = Q_Network(self.state_size, self.action_size).to(device)
        self.soft_update(1)
        self.optimizer = optim.Adam(self.Q_local.parameters(), self.lr)
        self.memory = deque(maxlen=100000)
        self.tst = None
        self.mu = [0]
        self.last_action = 0

    def act(self, state, eps=0):
        if random.random() > eps:
            state = torch.tensor(state, dtype=torch.float32).to(self.device)
            with torch.no_grad():
                action_values = self.Q_local(state).reshape(-1)
            if (action_values).max() > np.max(self.mu):
                # self.mu = 0.95*self.mu + 0.05*action_values.max()

                self.mu.append(action_values.max().cpu().data.numpy())                
                if len(self.mu) > 10:
                    self.mu = self.mu[-10:]
                self.last_action = np.argmax(action_values.cpu().data.numpy())
                return self.last_action
            else:
                return self.last_action
        else:
            action = random.choice(np.arange(self.action_size))
            self.last_action = action
            return action

    def learn(self):
        experiences = random.sample(self.memory, self.bs)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float().to(self.device)
        self.tst = states
        Q_values = self.Q_local(states).reshape(-1,11)
        Q_values = torch.gather(input=Q_values, dim=-1, index=actions)
        with torch.no_grad():
            Q_targets = self.Q_target(next_states)
            Q_targets, _ = torch.max(input=Q_targets, dim=-1, keepdim=True)
            Q_targets = rewards + self.gamma * (1 - dones) * Q_targets

        loss = (Q_values - Q_targets).pow(2).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def soft_update(self, tau):
        for target_param, local_param in zip(self.Q_target.parameters(), self.Q_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

# Train and Test

In [7]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 500
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.997
eps_min = 0.01
C = 4 # update weights every C steps

def validation(env, agent):
    # agent.mu=0
    env.mu=[0]
    rewards_log = []
    average_log = []
    episodic_reward = 0
    done = False
    frame = env.reset()
    state = frame
    t = 0
    while not done and t < max_t:
        t += 1
        action = agent.act(state, eps)
        frame, reward, done = env.step(action)
        rewards_log.append(reward)
        episodic_reward += reward
    sharpe = qs.stats.sharpe(pd.DataFrame(rewards_log))
    return env.asset, episodic_reward, sharpe


def train(env, agent, num_episode, eps_init, eps_decay, eps_min, max_t, num_frame=1, constant=0):
    # global rewards_log, average_log, state_history, action_history, done_history, reward_history
    rewards_log = []
    average_log = []
    state_history = []
    action_history = []
    done_history = []
    reward_history = []
    validation_log = []
    validation_average_log = []
    sharpe_log = []
    average_sharpe = []
    eps = eps_init
    for i in range(1, 1 + num_episode):
        env.mu=[0]
        episodic_reward = 0
        done = False
        frame = env.reset()
        state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        state = np.expand_dims(state, axis=0)
        t = 0

        while not done and t < max_t:

            t += 1
            action = agent.act(state, eps)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            next_state = np.expand_dims(next_state, axis=0)
            agent.memory.append((state, action, reward, next_state, done))

            if t % 5 == 0 and len(agent.memory) >= agent.bs:
                agent.learn()
                agent.soft_update(agent.tau)

            state = next_state.copy()
            episodic_reward += reward

        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        val_asset, val_reward, val_sharpe = validation(env_test, agent)

        validation_log.append(val_reward)
        validation_average_log.append(np.mean(validation_log[-100:]))
        sharpe_log.append(val_sharpe.values[0])
        average_sharpe.append(np.mean(sharpe_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}, valReward {:.3f}, val Average Reward {:.3f}, Asset {:.2f}, Validation Asset {:.2f}, Average Validation Sharpe {:.2f}'.format(i, episodic_reward, average_log[-1], val_reward, validation_average_log[-1], env.asset, val_asset, average_sharpe[-1]), end='')
        # print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}'.format(i, episodic_reward, average_log[-1]), end='')
        if i % 100 == 0:
            print()

        eps = max(eps * eps_decay, eps_min)

    return rewards_log

In [8]:
for code in codes:
    print(code, ' Begins')
    print('---------------------------------------------')
    env = Stock_Env(1000000, stock_df_train, 0.001, time = [x[0] for x in stock_df_train.index], record = stock_df_train_, codes_dict=codes_dict, train=True, code=code, time_period = time_period, codes=codes)
    env_test = Stock_Env(1000000, stock_df_test, 0.001, time = [x[0] for x in stock_df_test.index], record = stock_df_test_, codes_dict=codes_dict, train=False, code=code, time_period = time_period,  codes=codes)
    agent = Agent(2*3, 11, 64, 0.001, 0.001, 0.99, 'cuda')
    train(env, agent, num_episode, eps, eps_decay, eps_min, max_t, num_frame=1, constant=C)

AAPL  Begins
---------------------------------------------
Episode 100, Reward 0.062, Average Reward 0.037, valReward 0.462, val Average Reward 0.267, Asset 1057783.72, Validation Asset 1567928.03, Average Validation Sharpe 1.93
Episode 200, Reward 0.065, Average Reward 0.040, valReward 0.447, val Average Reward 0.275, Asset 1061808.40, Validation Asset 1546606.59, Average Validation Sharpe 1.98
Episode 300, Reward 0.115, Average Reward 0.057, valReward 0.286, val Average Reward 0.264, Asset 1115427.41, Validation Asset 1320320.42, Average Validation Sharpe 1.94
Episode 400, Reward 0.007, Average Reward 0.057, valReward 0.201, val Average Reward 0.273, Asset 1006620.31, Validation Asset 1208987.13, Average Validation Sharpe 1.98
Episode 500, Reward -0.002, Average Reward 0.063, valReward 0.318, val Average Reward 0.279, Asset 997702.20, Validation Asset 1360210.87, Average Validation Sharpe 2.05
AMZN  Begins
---------------------------------------------
Episode 100, Reward 0.014, Avera